In [1]:
import os
import functools
import pickle
import sys
from typing import Iterator, Generator

# import music21
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
%matplotlib inline

In [2]:
# Load data
score_names = np.load('./data/score_names.npy')

In [3]:
scores = [
    (np.load('./data/{}.npy'.format(i)))
    for i in tqdm(range(len(score_names)))
]

100%|██████████| 5930/5930 [00:10<00:00, 571.36it/s]


In [4]:
max_pitch = np.max([np.max(t) for t in scores])
min_pitch = np.min([np.min(t[t > 0]) for t in scores])
n_notes = int(max_pitch - min_pitch) + 1

In [5]:
from keras.preprocessing.text import one_hot

Using TensorFlow backend.


In [6]:
def make_targets(score, voice):
    """
    Make our target variables. It is the a stream of notes and one of metadata
    for a specified voice in the score.
    """
    n_output_features = n_notes
    y = np.zeros((score.shape[1], n_output_features))  # shape: n timesteps X m features
    for i, note in enumerate(score[voice]):
        if note > 0:
            note_idx = int(note - min_pitch)
            y[i, note_idx] = 1
    return y

In [7]:
def make_targets_meta(meta, voice):
    """
    Make our target variables. It is the a stream of notes and one of metadata
    for a specified voice in the score.
    """
    y_meta = np.zeros((meta.shape[1], 2)) #  add 2 meta features: slur, rest
    for i in range(meta.shape[1]):
        y_meta[i, idx_rest] = meta[voice, i, idx_rest]
        y_meta[i, idx_slur] = meta[voice, i, idx_slur]
    return y_meta

In [25]:
def make_padded(score, window_size, max_voices=None):
    # pad the beginning of the sequence so that our first window ends on the first timestep
    # also padd the voices
    padding_size = window_size - 1
    if max_voices is not None:

        voices_padding_size = max_voices - score.shape[0]
        voices_padding = np.zeros((voices_padding_size, score.shape[1]))
        score = np.vstack((score, voices_padding))
        score_padding = np.zeros((max_voices, padding_size))
    else:
        score_padding = np.zeros((score.shape[0], padding_size))
    return np.hstack((score_padding, score))


In [26]:
def make_padded_meta(meta, window_size):
    padding_size = window_size - 1
    meta_padding = np.zeros((meta.shape[0], padding_size, meta.shape[2]))
    return np.hstack((meta_padding, meta))

In [27]:
def make_input_sequence(score, voice, sequence_steps=16, conv_window_size=32):
    """
    Make an input sequence for a particular voice
    """
    window_size = sequence_steps * conv_window_size
    padded_score = make_padded(score, window_size)
    padding_size = window_size - 1
    indexer = np.arange(window_size)[None, :] + np.arange(padded_score.shape[1] - padding_size)[:, None]
    score_sequence = padded_score.T[indexer, :, None]

    # Now, mask out the target values
    score_sequence[:, -1, voice, :] = 0
    
    return score_sequence.reshape((score.shape[1], -1, conv_window_size, padded_score.shape[0], 1)) / max_pitch

In [28]:
def make_input_sequence_meta(meta, voice, sequence_steps=16, conv_window_size=32):
    window_size = sequence_steps * conv_window_size
    padded_meta = make_padded_meta(meta, window_size)
    
    padding_size = window_size - 1
    indexer = np.arange(window_size)[None, :] + np.arange(padded_score.shape[1] - padding_size)[:, None]
    meta_sequence = np.swapaxes(padded_meta, 0, 1)[indexer, :, :]
    
    # Now, mask out the target values
    meta_sequence[:, -1, voice, :] = 0
    
    return meta_sequence

In [29]:
np.random.seed(25)

In [30]:
scores_train, scores_valid = train_test_split(scores, test_size=0.1)

In [31]:
scores_valid[0].shape

(5, 880)

In [32]:
def cycle(iterable):
    while True:
        for i in iterable:
            yield i

In [33]:
# hyperparameters
SEQUENCE_STEPS = 4
window_size = 32
n_features = 58

In [34]:
train_gen = cycle(
    (make_input_sequence(score, voice, sequence_steps=SEQUENCE_STEPS), make_targets(score, voice))
    for score in scores_train
    for voice in range(score.shape[0])
)

In [35]:
make_input_sequence(scores_valid[0], 0).shape

(880, 16, 32, 5, 1)

In [36]:
valid_gen = cycle(
    (make_input_sequence(score, voice, sequence_steps=SEQUENCE_STEPS), make_targets(score, voice))
    for score in scores_valid
    for voice in range(score.shape[0])
)

In [37]:
next(train_gen)[0].shape

(2160, 4, 32, 6, 1)

In [39]:
from keras.models import Sequential, Model
from keras import layers
from keras.activations import relu
import keras.callbacks

In [42]:

# model = Sequential()
# model.add(TimeDistributed(layers.Conv2D(32, 3, padding='same', activation='relu'), input_shape=(SEQUENCE_STEPS, window_size, 6, 1)))
# model.add(TimeDistributed(layers.Conv2D(32, 3, padding='same', activation='relu')))
# model.add(TimeDistributed(layers.MaxPool2D(2, 2, padding='same')))
# # model.add(TimeDistributed(Conv2D(64, 3, padding='same', activation='relu')))
# model.add(TimeDistributed(layers.Conv2D(64, 3, padding='same', activation='relu')))
# model.add(TimeDistributed(layers.MaxPool2D(2, 2, padding='same')))
# model.add(TimeDistributed(layers.Conv2D(128, 3, padding='same', activation='relu')))
# model.add(TimeDistributed(layers.Dropout(0.1)))
# model.add(TimeDistributed(layers.GlobalAveragePooling2D()))
# model.add(LSTM(128, return_sequences=True, unroll=True, dropout=0.1))
# model.add(LSTM(128, dropout=0.1, unroll=True))
# # model.add(Dense(200, activation='relu'))
# model.add(Dense(n_features, activation='softmax'))

model = Sequential()
model.add(layers.Conv3D(32, 3, padding='same', activation='relu', input_shape=(SEQUENCE_STEPS, window_size, None, 1)))
model.add(layers.Conv3D(32, 3, padding='same', activation='relu'))
model.add(layers.max)

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_3 (Conv3D)            (None, 4, 32, None, 32)   896       
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 4, 32, None, 32)   27680     
Total params: 28,576
Trainable params: 28,576
Non-trainable params: 0
_________________________________________________________________


In [120]:
top3_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=3)

top3_acc.__name__ = 'top3_acc'



In [121]:
model.compile('adam', loss='categorical_crossentropy', metrics=['accuracy', top3_acc])

In [122]:
checkpointer = keras.callbacks.ModelCheckpoint(filepath='./models/model5.hdf5', verbose=1, save_best_only=True)

In [123]:
steps_per_epoch = np.sum(score.shape[0] for score in scores_train)
steps_per_epoch

25420

In [124]:
validation_steps = np.sum(score.shape[0] for score in scores_valid)
validation_steps

2835

In [125]:
try:
    model.load_weights('./models/model5.hdf5')
except OSError:
    print('no weights found')

no weights found


In [ ]:
history = model.fit_generator(
    train_gen,
    steps_per_epoch=steps_per_epoch,
    epochs=5,
    validation_data=valid_gen,
    validation_steps=validation_steps,
    callbacks=[checkpointer]
)

Epoch 1/5
  228/25420 [..............................] - ETA: 1:19:51 - loss: 2.8916 - acc: 0.0475 - top3_acc: 0.1559

In [ ]:
model.evaluate_generator(valid_gen, steps=validation_steps)